In [13]:
import codecs,unidecode,yaml,glob,os,shutil,re
import itertools as it
import numpy as np
import pandas as pd
print(pd.__version__)

0.24.2


In [34]:
nQuizz=5
fMajeure=u"/Users/gilles/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/01-NotesQuizz/Majeure.csv"
fMineure=u"/Users/gilles/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/01-NotesQuizz/Mineure.csv"
fCM=u"/Users/gilles/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/01-NotesQuizz/CM.csv"
fAliasQDLE=u"/Users/gilles/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/01-NotesQuizz/Quizz-Alias.yaml"
fQuizzoodle=u"/Users/gilles/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/01-NotesQuizz/Quizz%02d.csv"%nQuizz
fNotes=fQuizzoodle.replace(u".csv",u"-Notes.csv")

In [27]:
'''https://stackoverflow.com/a/35491326/3986209'''
def invert_dict(d): 
    inverse = dict() 
    for key in d: 
        # Go through the list that is saved in the dict:
        for item in d[key]:
            # Check if in the inverted dict the key exists
            if item not in inverse: 
                # If not create a new list
                inverse[item] = [key] 
            else: 
                inverse[item].append(key) 
    return inverse

# Lecture des données

In [28]:
try:
    with codecs.open(fAliasQDLE, 'r', encoding="utf8") as inFile:
        correspondances=yaml.safe_load(inFile)
    alias=correspondances["alias2etu"]
except:
    alias={}
alias

{'APPADOO et VICENTINI': ['412653', '415269'],
 'Aaron Dieu': ['413093'],
 'Abigail raguraj': ['397598'],
 'Afonso Peleja': ['415773'],
 'Agathe Corbier': ['398357'],
 'Agleizes001': ['412341'],
 'Aiden Bertin': ['415098'],
 'Alexis peyen': ['414807'],
 'Alice Dupouy': ['400421'],
 'Alicia PARISET': ['397359'],
 'Alicia PARISET Axel Hennebert': ['397359', '414202'],
 u'Aliz\xe9e Artaud': ['415398'],
 'Amblard Juline': ['415539'],
 'Ambre Sovieri': ['402017'],
 'Amelie Martin': ['398003'],
 'Amine elhaloui': ['415818'],
 u'Am\xe9lie Martin': ['398003'],
 'Anais Bernou': ['413687'],
 u'Ana\xefs AURIAULT': ['418584'],
 u'Ana\xefs Bernou': ['413687'],
 'Angele': ['417705'],
 'Anne-Emmanuelle': ['421063'],
 'Appadoo et Vicentini': ['415269', '412653'],
 'Aude-Lise': ['398735'],
 'Aude-Lise Ledentu': ['398735'],
 'Aure girard': ['414227'],
 'Aurel@700': ['416671'],
 'Aurelm7000': ['416671'],
 'Axel Hennebert': ['414202'],
 u'BARNOLE Myl\xe9nie': ['413653'],
 'BENZONI Enzo': ['414854'],
 'BOT

In [5]:
len(alias)

366

In [35]:
colCM=[u"id",u"nom",u"prénom",u"email",u"ine"]
colQuizz=["pseudo","points","temps"]

try:
    dfCM=pd.read_csv(filepath_or_buffer=fCM,sep="\t",encoding="utf8",header=0,names=colCM)
    lQuizz=pd.read_csv(filepath_or_buffer=fQuizzoodle,sep=";",encoding="utf8",header=0,names=colQuizz)
    lQuizz.pseudo=lQuizz.pseudo.str.strip()
except:
    dfCM=None
    lQuizz=None

In [36]:
lQuizz

,pseudo,points,temps
0,VSR,7,43.54
1,Théo,7,44.28
2,Alexis peyen,7,63.60
3,DA CUNHA Mathilde,7,129.81
4,Nawell Bachani,7,133.47
5,Piveteau Océane,7,160.27
6,Maxine Martineau,7,164.66
7,Amelie Martin,7,167.87
8,Y.auger,7,168.33
9,Justine A,7,191.11


# Listage des alias

In [ ]:
dfCM["PN"]=dfCM[u"prénom"].str.lower().str.strip()+" "+dfCM[u"nom"].str.lower().str.strip()
dfCM["NP"]=dfCM[u"nom"].str.lower().str.strip()+" "+dfCM[u"prénom"].str.lower().str.strip()
dfCM

In [ ]:
pseudos=lQuizz[~(lQuizz["pseudo"].isin(alias))]["pseudo"].str.strip().tolist()

i=39
for nP,pseudo in enumerate(pseudos[i:i+1]):
    print pseudo
    sPseudo = raw_input("Nom a chercher:\n")
    print dfCM[dfCM["PN"].str.contains(sPseudo)][["PN","id"]]

In [ ]:
print pseudo
alias[pseudo]=["416125"]

In [ ]:
len(pseudos)

In [30]:
alias["Sofiane"]

KeyError: 'Sofiane'

In [ ]:
pn=dfCM["PN"].tolist()
np=dfCM["NP"].tolist()

# "damien guerrero" in pn

In [ ]:
for pseudo in pseudos:
    if unidecode.unidecode(pseudo.lower()) in pn:
        print pseudo,dfCM[dfCM["PN"]==unidecode.unidecode(pseudo.lower())]["id"].values[0]
        alias[pseudo]=[str(dfCM[dfCM["PN"]==unidecode.unidecode(pseudo.lower())]["id"].values[0])]
    elif unidecode.unidecode(pseudo.lower()) in np:
        print pseudo,dfCM[dfCM["NP"]==unidecode.unidecode(pseudo.lower())]["id"].values[0]
        alias[pseudo]=[str(dfCM[dfCM["NP"]==unidecode.unidecode(pseudo.lower())]["id"].values[0])]



In [ ]:
for al in alias:
    if not isinstance(alias[al],list):
        alias[al]=[alias[al]]

In [ ]:
len(alias)

In [ ]:
invAlias=invert_dict(alias)
#invAlias[385732]=[u'adiarra001, cbeauvais004']
correspondances={"alias2etu":alias,"etu2alias":invAlias}
correspondances

In [ ]:
with codecs.open(fAliasQDLE, 'w', encoding="utf8") as outfile:
    yaml.safe_dump(correspondances, outfile, default_flow_style=False)

# Calcul des notes

In [7]:
nbQ=7.0

In [8]:
def aliasQuizz(pseudo):
    result=None
    if pseudo in alias:
        result=alias[pseudo]
    return result

def noteQuizz(points):   
    result=8
    if points>0:
        result=int(round(10*(1+points/nbQ),0))
    return result
        
        

lQuizz["id"]=lQuizz["pseudo"].apply(aliasQuizz)
lQuizz["note"]=lQuizz["points"].apply(noteQuizz)

In [9]:
def splitPseudo1(id):
    result=None
    if id and len(id)>0:
        result=id[0]
    return result

def splitPseudo2(id):
    result=None
    if id and len(id)>1:
        result=id[1]
    return result

def splitPseudo3(id):
    result=None
    if id and len(id)>2:
        result=id[2]
    return result

lQuizz["id1"]=lQuizz["id"].apply(splitPseudo1)
lQuizz["id2"]=lQuizz["id"].apply(splitPseudo2)
lQuizz["id3"]=lQuizz["id"].apply(splitPseudo3)

lQuizz

,pseudo,points,temps,id,note,id1,id2,id3
0,Théo,7,44.28,None,20,None,None,None
1,Alexis peyen,7,63.60,[414807],20,414807,None,None
2,DA CUNHA Mathilde,7,129.81,[415439],20,415439,None,None
3,Nawell Bachani,7,133.47,[414683],20,414683,None,None
4,Piveteau Océane,7,160.27,[412145],20,412145,None,None
5,Maxine Martineau,7,164.66,[413026],20,413026,None,None
6,Amelie Martin,7,167.87,[398003],20,398003,None,None
7,Y.auger,7,168.33,[415389],20,415389,None,None
8,Justine A,7,191.11,None,20,None,None,None
9,Colline Cherel,6,60.69,[411946],19,411946,None,None


In [10]:
df1=lQuizz[["pseudo","note","temps","id1"]].dropna()
df2=lQuizz[["pseudo","note","temps","id2"]].dropna()
df3=lQuizz[["pseudo","note","temps","id3"]].dropna()
df1.columns=["pseudo","note","temps","id"]
df2.columns=["pseudo","note","temps","id"]
df3.columns=["pseudo","note","temps","id"]

In [11]:
df123=pd.concat([df1,df2,df3]).sort_values(["note","temps"],ascending=[False,True])
df123

,pseudo,note,temps,id
1,Alexis peyen,20,63.60,414807
2,DA CUNHA Mathilde,20,129.81,415439
3,Nawell Bachani,20,133.47,414683
4,Piveteau Océane,20,160.27,412145
5,Maxine Martineau,20,164.66,413026
6,Amelie Martin,20,167.87,398003
7,Y.auger,20,168.33,415389
9,Colline Cherel,19,60.69,411946
10,Meunier Theo,19,64.29,414747
11,Carla Vicentini,19,71.91,415269


In [12]:
df123.groupby("id").max().to_csv("TEMP.CSV",sep="\t",encoding="utf8")